In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = "../input/face-expression-recognition-dataset/images/train"
val_dir = "../input/face-expression-recognition-dataset/images/validation"

In [3]:
batch_size=64
img_height=180
img_width=180
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, 
                                                      validation_split = 0.2,
                                                      subset = 'training',
                                                      seed = 123,
                                                      image_size = (img_height, img_width),
                                                      batch_size = batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(val_dir,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    seed=123,
                                                    image_size=(img_height, img_width),
                                                    batch_size=batch_size)

In [4]:
"""
batch_size = 32
img_height = 150
img_width = 150
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
train_ds = train_datagen.flow_from_directory(train_dir,
                                            target_size = (img_height, img_width),
                                            batch_size = batch_size,
                                            class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1./255)
val_ds = val_datagen.flow_from_directory(val_dir,
                                         target_size=(img_height, img_width),
                                         batch_size=batch_size,
                                         class_mode='categorical') """

In [5]:
class_names = train_ds.class_names
print(class_names)

# Data Visualization

In [6]:
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

In [7]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [8]:
""""
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE) """

### Early Stopping

In [9]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

### Data augmentation

In [10]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                   img_width,
                                   3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [11]:
tf.keras.backend.clear_session()

## Create, compile and train the Model

In [25]:
num_classes = 7

model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(256,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(512,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(512,(3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation = 'softmax')
])


In [26]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [27]:
epochs=80
history = model.fit(train_ds, validation_data = val_ds,
                    epochs= epochs,
                    callbacks=[callback])

## Visualize training results

In [28]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()